Dla ograniczen: <br />
1.Ograniczenia dostępności pracowników według tabeli <br />
2.Ograniczenie minimalnej liczby osób danego dnia <br />
3.Ograniczenia minimalnej i maksymalnej liczby przepracownych dni pracowników, w tygodniu (min 1, max 3)

| Imię       | dzien tygodnia    | płaca   |
|------------|-------------------|---------|
| ANIA       | kazdy             | 100/h   |
| STEFAN     | pon               | 50/h    |
| HEKTOR     | pon, wto, sro     |  60/h   |
| OLAF       | kazdy             | 40/h    |
| LIDIA      | czw, pio          |  110/h  |
| PIOTR      | pon, wto, sro     |  70/h   |



| pon  | wto    | sro   | czwa  | pio   |
|------|--------|-------|-------|-------|
| 2    | 1      | 1     |  1    | 3     |


Stworzyć taki grafik, aby wartośc wypłat była jak najniższa 

In [ ]:
!pip install pulp
!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [5]:
import pulp
print(pulp.__version__)

2.3.1


In [4]:
from pulp import *
import pandas as pd

pracownicy = {
    'ANIA':{'dni':['pn','wt','sr','czw','pt'],'placa': 100},
    'STEFAN':{'dni':['pn'],'placa': 50},
    'HEKTOR':{'dni':['pn','wt','sr'],'placa': 60},
    'OLAF':{'dni':['pn','wt','sr','czw','pt'],'placa': 40},
    'LIDIA':{'dni':['czw','pt'],'placa': 110},
    'PIOTR':{'dni':['pn','wt','sr'],'placa': 70},
}

liczbaPracownikow = {
    'pn': 2,
    'wt': 1,
    'sr': 1,
    'czw': 1,
    'pt': 3
}
tydzien = ['pn','wt','sr','czw','pt']

vars = LpVariable.dicts('x', (pracownicy.keys(), tydzien),
                            lowBound = 0,
                            upBound = 1,
                            cat = 'Integer')



prob = LpProblem("Praca_Model", LpMinimize)

prob += 4*lpSum(pracownicy[pracownik]['placa']*lpSum(vars[pracownik][dzien] for dzien in tydzien) for pracownik in pracownicy)


for pracownik in pracownicy:
  temp = 0
  for dzien in tydzien:
    if dzien not in pracownicy[pracownik]["dni"]:
      temp += vars[pracownik][dzien]
  prob += temp == 0

for dzien in tydzien:
  prob += lpSum(vars[pracownik][dzien] for pracownik in pracownicy) >= liczbaPracownikow[dzien]

for pracownik in pracownicy:
  prob += lpSum(vars[pracownik][dzien] for dzien in tydzien) <= 3
  prob += lpSum(vars[pracownik][dzien] for dzien in tydzien) >= 1

prob.writeLP("schedule.lp")
prob.solve()

print("Status:", LpStatus[prob.status])
print()
print("Minimalna wartośc wypłat:", value(prob.objective), "PLN",)



Status: Optimal

Minimalna wartośc wypłat: 2040.0 PLN
